In [1]:
import json
import os
import pandas as pd 
import numpy as np
from datetime import date

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
file_date = date.today().strftime("%Y%m%d")

In [4]:
with open("vehicle_position.json", "r") as read_file:
    data = json.load(read_file)

In [5]:
entities = data['entity']
entities[0]

{'id': '1',
 'vehicle': {'trip': {'tripId': '1--L.1648.101.8.B.8.61298061',
   'scheduleRelationship': 'SCHEDULED',
   'routeId': 'IWL_2d'},
  'position': {'latitude': -33.86420440673828,
   'longitude': 151.04629516601562},
  'timestamp': '1586842838',
  'congestionLevel': 'UNKNOWN_CONGESTION_LEVEL',
  'stopId': 'Lidcombe.Lidcombe 3 Loc',
  'vehicle': {'id': '2257.4727.6737.6076.1001.6798.1029.9418',
   'label': '14:54 Leppington Station to Central Station '}}}

In [6]:
entities[0]['vehicle']['vehicle']['label']
if 'label' in entities[0]['vehicle']['vehicle']:
    print('yes')

yes


In [7]:
entities[0]['vehicle']['trip']
if 'routeId' in entities[0]['vehicle']['trip']:
    print('yes')

yes


In [8]:
vehicle = []
tripId = []
stopId = []
scheduleRelationship = []
routeId = []
latitude= []
longitude = []
timestamp = []
congestionLevel = []
label = []
for entity in entities:
    if 'id' in entity['vehicle']['vehicle']:
        vehicle.append(entity['vehicle']['vehicle']['id'])
    else:
        vehicle.append("")
    tripId.append(entity['vehicle']['trip']['tripId'])
    stopId.append(entity['vehicle']['stopId'])
    if 'routeId' in entity['vehicle']['trip']:
        routeId.append(entity['vehicle']['trip']['routeId'])
    else:
        routeId.append("")
    scheduleRelationship.append(entity['vehicle']['trip']['scheduleRelationship'])
    latitude.append(entity['vehicle']['position']['latitude'])
    longitude.append(entity['vehicle']['position']['longitude'])
    timestamp.append(entity['vehicle']['timestamp'])
    congestionLevel.append(entity['vehicle']['congestionLevel'])
    if 'label' in entity['vehicle']['vehicle']:
        label.append(entity['vehicle']['vehicle']['label'])
    else:
        label.append("")

In [9]:
df = pd.DataFrame(columns = ['date', 'time', 'timestamp','vehicle','tripId','stopId','scheduleRelationship','routeId',
                            'latitude','longitude','congestionLevel','label'])

df['vehicle'] = pd.Series(vehicle)
df['tripId'] = pd.Series(tripId)
df['stopId'] = pd.Series(stopId)
df['scheduleRelationship'] = pd.Series(scheduleRelationship)
df['routeId'] = pd.Series(routeId)
df['latitude'] = pd.Series(latitude)
df['longitude'] = pd.Series(longitude)
df['timestamp'] = pd.Series(timestamp)
df['congestionLevel'] = pd.Series(congestionLevel)
df['label'] = pd.Series(label)

In [10]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.timestamp = df.timestamp + pd.Timedelta('10:00:00')
df['date'] = pd.to_datetime(df['timestamp']).dt.date
df['time'] = pd.to_datetime(df['timestamp']).dt.time

In [11]:
df.drop('timestamp', axis=1, inplace=True)

In [12]:
# Drop the rows of Nsw Trains
df = df[~(df["routeId"].isin(['BMT_1', 'BMT_2', 'CCN_1a', 'CCN_1b', 'CCN_1c',
                               'CCN_2a', 'CCN_2b', 'CTY_NC1', 'CTY_NC1a',
                               'CTY_NC2', 'CTY_NW1a', 'CTY_NW1b', 'CTY_NW1c',
                               'CTY_NW1d', 'CTY_NW2a', 'CTY_NW2b', 'CTY_S1a',
                               'CTY_S1b', 'CTY_S1c', 'CTY_S1d', 'CTY_S1e',
                               'CTY_S1f', 'CTY_S1g', 'CTY_S1h', 'CTY_S1i',
                               'CTY_S2a', 'CTY_S2b', 'CTY_S2c', 'CTY_S2d',
                               'CTY_S2e', 'CTY_S2f', 'CTY_S2g', 'CTY_S2h',
                               'CTY_S2i', 'CTY_W1a', 'CTY_W1b', 'CTY_W2a',
                               'CTY_W2b', 'HUN_1a', 'HUN_1b', 'HUN_2a',
                               'HUN_2b', 'SCO_1a', 'SCO_1b', 'SCO_2a',
                               'SCO_2b', 'SHL_1a', 'SHL_1b', 'SHL_1c',
                               'SHL_1d', 'SHL_1e', 'SHL_2a', 'SHL_2b', 'SHL_2c',
                               'SHL_2d', 'SHL_2e', 'RTTA_DEF', 'RTTA_REV']))]

In [13]:
df = df[~df.tripId.str.contains("NonTimetabled")]

In [14]:
hdr = False if os.path.isfile('vehicle_position_' + file_date + '.csv') else True
df.to_csv('vehicle_position_' + file_date + '.csv', mode='a', header=hdr, index=False)